Import libraries

In [114]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


Import csv file

In [115]:
data = pd.read_csv('./spam.csv') 

ilk 10 veriyi ekrana yazdıralım.

In [116]:
data.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [118]:
# Geriye tuple formatında satır sayısı ve kolon sayısını döndürür.
data.shape

(5572, 2)

Kategorik verinin sayısal veriye dönüştürüldüğü aşama, ham = 0, spam = 1 olarak set edildi.

In [119]:
# 'Category' sütunu 'ham' değerine sahip olan satırların 'Category' sütunundaki değeri 0 olarak değiştiriliyor.
data.loc[data['Category'] == 'ham', 'Category',] = 0
# 'Category' sütunu 'spam' değerine sahip olan satırların 'Category' sütunundaki değeri 1 olarak değiştiriliyor.
data.loc[data['Category'] == 'spam', 'Category',] = 1
print(data.head(10))

  Category                                            Message
0        0  Go until jurong point, crazy.. Available only ...
1        0                      Ok lar... Joking wif u oni...
2        1  Free entry in 2 a wkly comp to win FA Cup fina...
3        0  U dun say so early hor... U c already then say...
4        0  Nah I don't think he goes to usf, he lives aro...
5        1  FreeMsg Hey there darling it's been 3 week's n...
6        0  Even my brother is not like to speak with me. ...
7        0  As per your request 'Melle Melle (Oru Minnamin...
8        1  WINNER!! As a valued network customer you have...
9        1  Had your mobile 11 months or more? U R entitle...


In [120]:
X_train, X_test, Y_train, Y_test = train_test_split(data['Message'], data['Category'], test_size=0.2, random_state=3)

In [121]:
print("Total mesaj sayısı: ", data["Message"].shape) # total sayısı
print("Traine ayrılan sayı: ", X_train.shape) # train'e ayrılan sayı
print("Teste ayrılan sayı: ", X_test.shape) # teste ayrılan sayı

Total mesaj sayısı:  (5572,)
Traine ayrılan sayı:  (4457,)
Teste ayrılan sayı:  (1115,)


Hiperparametreleri ayarladım.

In [122]:
# stop_words='english' en sık geçen kelimeleri ('the', 'a', 'an', etc.) kaldırır.
vectorizer = TfidfVectorizer(stop_words='english')
# eğitim verilerini sayısal özellik vektörlerine dönüştürüyoruz.
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [123]:
# Y train ve test verilerini int'e çeviriyoruz.
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [124]:
# Grid Search için parametreleri belirliyoruz.
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
logistic_regression = LogisticRegression()
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_features, Y_train)

c:\Users\ismail\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='accuracy')

In [125]:
# En iyi parametreleri kullanarak bir Lojistik Regresyon modeli oluşturuluyor.
best_logreg_model = LogisticRegression(C=grid_search.best_params_['C'])
# Eğitim verileri üzerinde Lojistik Regresyon modeli eğitiliyor.
best_logreg_model.fit(X_train_features, Y_train)

# Eğitim verileri üzerinde tahmin yapılıyor.
prediction_on_training_data = best_logreg_model.predict(X_train_features)
# Eğitim doğruluğu hesaplanıyor.
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

# Test verileri üzerinde tahmin yapılıyor.
prediction_on_test_data = best_logreg_model.predict(X_test_features)
# Test doğruluğu hesaplanıyor.
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)
print(accuracy_on_test_data)


0.9820627802690582


In [126]:
from sklearn.metrics import mean_squared_error
from math import sqrt
MSE=mean_squared_error(Y_test,prediction_on_test_data)
print("Mean Squared Error:",MSE)
RMSE=sqrt(MSE)
print("Root Mean Squared Error:",RMSE)

Mean Squared Error: 0.017937219730941704
Root Mean Squared Error: 0.133929906036485
